# PySpark Tutorial

## Section 1: Spark I/O with `pyspark.sql` module

### `SparkContext` and `SparkSession`

For a pyspark to run, a `sc` is needed to establish a connection to the Spark cluster (similar to a `sqlalchemy` database Engine), while a `spark` session is needed to **interact** with `sc`. Often times, there's a need for user to ensure only **1** `sc` and `spark` exist when running to avoid serious bugs.

In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

### `SparkConf` is used to define the meta-settings for a SparkContext

In [2]:
conf = SparkConf().setMaster('local').setAppName('SparkBeginner')
sc = SparkContext(conf=conf)

`sc.version` indicates the specific Spark version of the cluster

In [3]:
print(sc)

<SparkContext master=local appName=SparkBeginner>


In [4]:
print(sc.version)

2.4.4


### Use `pyspark.sql` to interact with PySpark DataFrame
It is often recommended to use the `pyspark.sql` API to interact with **PySpark DataFrame**, which is a higher level API than the operations around Spark RDDs. To use the `pyspark.sql` API, a `SparkSession` is always needed.

In [5]:
# using .getOrCreate() method of the builder will only create a new 
# spark session unless there's none such session existing
my_spark = SparkSession.builder.getOrCreate()

In [6]:
print(my_spark)

Your `SparkSession` has an attribute called `catalog` which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information. One of the most useful is the `.listTables()` method, which returns the names of all the tables in your cluster as a list.

In [7]:
my_spark.catalog.listTables()

[]

### Load and Save Data into Spark

Let's load a csv table into Spark local view. The `spark.read.methods()` provides an API to reading different file formats into `sc`.

In [8]:
links_df = my_spark.read.csv('./input/ml-latest-small/links.csv', header=True)

In [9]:
# load all the other movielens tables into spark context
movies_df = my_spark.read.csv('./input/ml-latest-small/movies.csv', header=True)
ratings_df = my_spark.read.csv('./input/ml-latest-small/ratings.csv', header=True)
tags_df = my_spark.read.csv('./input/ml-latest-small/tags.csv', header=True)

In [10]:
links_df.show()

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
|     11|0112346|  9087|
|     12|0112896| 12110|
|     13|0112453| 21032|
|     14|0113987| 10858|
|     15|0112760|  1408|
|     16|0112641|   524|
|     17|0114388|  4584|
|     18|0113101|     5|
|     19|0112281|  9273|
|     20|0113845| 11517|
+-------+-------+------+
only showing top 20 rows



Calling `.createOrReplaceTempView('table_name')` will create a table name inside the SparkContext. **Notice that until creating a "View", the imported data will not show up in `spark.catalog.listTables()`.**

In [12]:
links_df.createOrReplaceTempView('links')
movies_df.createOrReplaceTempView('movies')
ratings_df.createOrReplaceTempView('ratings')
tags_df.createOrReplaceTempView('tags')

In [13]:
my_spark.catalog.listTables()

[Table(name='links', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='movies', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='tags', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

To use this dataframe via `pandas`, simply call `.toPandas()` method on the Spark DataFrame

In [14]:
links_pd = links_df.toPandas()

In [15]:
links_pd

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862
...,...,...,...
9737,193581,5476944,432131
9738,193583,5914996,445030
9739,193585,6397426,479308
9740,193587,8391976,483455


Alternatively, one can easily turn any pandas dataframe into a spark dataframe with `spark.createDataFrame()`

In [16]:
temp_df = pd.DataFrame({'X': [1, 2, 3],
                        'y': [0, 0, 1]})
temp_spdf = my_spark.createDataFrame(temp_df)

In [17]:
temp_spdf.show()

+---+---+
|  X|  y|
+---+---+
|  1|  0|
|  2|  0|
|  3|  1|
+---+---+



To read a table from Spark, simply call `spark.table('table_name')`

In [18]:
links = my_spark.table('links')

In [19]:
links.show()

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
|      6|0113277|   949|
|      7|0114319| 11860|
|      8|0112302| 45325|
|      9|0114576|  9091|
|     10|0113189|   710|
|     11|0112346|  9087|
|     12|0112896| 12110|
|     13|0112453| 21032|
|     14|0113987| 10858|
|     15|0112760|  1408|
|     16|0112641|   524|
|     17|0114388|  4584|
|     18|0113101|     5|
|     19|0112281|  9273|
|     20|0113845| 11517|
+-------+-------+------+
only showing top 20 rows



## Section 2: Data Manipulation with PySpark

### Use `spark_df.withColumn()` to conduct column-wise operations

For column-wise operations, use the `spark_dataframe.withColumn()` method that takes two arguments:

1. first arg, a string with the name of your new column
2. second arg, the new column itself (i.e. the calculation)

The new column in arg 2 must be an object of class `Column`. Creating one of these is as easy as extracting a column from your DataFrame using `spark_dataframe.colName`.

**NOTE:**

Updating a Spark DataFrame is different than working in pandas because the Spark DataFrame is ***immutable***. This means that it can't be changed, and so columns can't be updated in place. Instead, all the operations methods will return a new DataFrame.

In order to overwrite the original DataFrame, we must reassign the returned DataFrame by assigning it back to the original spark dataframe name:

```python
df = df.withColumn("newCol", df.oldCol * 2)
```

The above code creates a DataFrame with the same columns as `df` **AND** a new column, newCol, where every entry is equal to the corresponding entry from oldCol multiplied by 2.

To overwrite an existing column, just pass the name of the column as the first argument!

In [26]:
ratings = my_spark.table('ratings')
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [27]:
# change the data format of the 'rating' column from float to int
ratings = ratings.withColumn('rating_int', ratings['rating'].astype('int'))
ratings.show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|rating_int|
+------+-------+------+---------+----------+
|     1|      1|   4.0|964982703|         4|
|     1|      3|   4.0|964981247|         4|
|     1|      6|   4.0|964982224|         4|
|     1|     47|   5.0|964983815|         5|
|     1|     50|   5.0|964982931|         5|
|     1|     70|   3.0|964982400|         3|
|     1|    101|   5.0|964980868|         5|
|     1|    110|   4.0|964982176|         4|
|     1|    151|   5.0|964984041|         5|
|     1|    157|   5.0|964984100|         5|
|     1|    163|   5.0|964983650|         5|
|     1|    216|   5.0|964981208|         5|
|     1|    223|   3.0|964980985|         3|
|     1|    231|   5.0|964981179|         5|
|     1|    235|   4.0|964980908|         4|
|     1|    260|   5.0|964981680|         5|
|     1|    296|   3.0|964982967|         3|
|     1|    316|   3.0|964982310|         3|
|     1|    333|   5.0|964981179|         5|
|     1|  

### Use `spark_df.filter()` to filter rows

The `.filter()` method takes in 2 formats for filtering:

1. A SQL string will work if it works on an equivalent `WHERE...` SQL statment clause
2. A boolean `Column` will work, similar to how normally a pandas selector works

In [39]:
# below 2 calls return the same results
ratings.filter('movieId == 1').show()
ratings.filter(ratings.movieId == 1).show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|rating_int|
+------+-------+------+----------+----------+
|     1|      1|   4.0| 964982703|         4|
|     5|      1|   4.0| 847434962|         4|
|     7|      1|   4.5|1106635946|         4|
|    15|      1|   2.5|1510577970|         2|
|    17|      1|   4.5|1305696483|         4|
|    18|      1|   3.5|1455209816|         3|
|    19|      1|   4.0| 965705637|         4|
|    21|      1|   3.5|1407618878|         3|
|    27|      1|   3.0| 962685262|         3|
|    31|      1|   5.0| 850466616|         5|
|    32|      1|   3.0| 856736119|         3|
|    33|      1|   3.0| 939647444|         3|
|    40|      1|   5.0| 832058959|         5|
|    43|      1|   5.0| 848993983|         5|
|    44|      1|   3.0| 869251860|         3|
|    45|      1|   4.0| 951170182|         4|
|    46|      1|   5.0| 834787906|         5|
|    50|      1|   3.0|1514238116|         3|
|    54|      1|   3.0| 830247330|

### Use `spark_df.select()` to **cherry pick** columns

The Spark variant of SQL's `SELECT` is the `spark_df.select()` method. This method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a string (one for each column) or a column object (using the `df.colName` syntax). When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside `.withColumn()`.

**NOTE:**
The difference between `.select()` and `.withColumn()` methods is that `.select()` returns ***only the columns you specify***, while `.withColumn()` returns ***all the columns of the DataFrame in addition to the one you defined***.

It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use `.select()` and not `.withColumn()`.

In [40]:
# below 2 sets of codes selects the same results
ratings.select('userId', 'movieId', 'rating').show()
ratings.select(ratings.userId, ratings.movieId, ratings.rating).show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    23

In [86]:
# use .withColumnRanamed if only column action is to rename column
ratings.withColumnRenamed('rating', 'movieRating').show()

+------+-------+-----------+---------+----------+
|userId|movieId|movieRating|timestamp|rating_int|
+------+-------+-----------+---------+----------+
|     1|      1|        4.0|964982703|         4|
|     1|      3|        4.0|964981247|         4|
|     1|      6|        4.0|964982224|         4|
|     1|     47|        5.0|964983815|         5|
|     1|     50|        5.0|964982931|         5|
|     1|     70|        3.0|964982400|         3|
|     1|    101|        5.0|964980868|         5|
|     1|    110|        4.0|964982176|         4|
|     1|    151|        5.0|964984041|         5|
|     1|    157|        5.0|964984100|         5|
|     1|    163|        5.0|964983650|         5|
|     1|    216|        5.0|964981208|         5|
|     1|    223|        3.0|964980985|         3|
|     1|    231|        5.0|964981179|         5|
|     1|    235|        4.0|964980908|         4|
|     1|    260|        5.0|964981680|         5|
|     1|    296|        3.0|964982967|         3|


It is also possible to do column-wise calculation with the `.select()` method. Methodology as below:

1. In pythonic API, define the new `Column` by calling the column operations and use `.alias()` to rename the column if needed:
```python
new_col = (df.old_col1 * 2 - df.old_col2).alias('new_col')
```
Then use the `.select()` as always:
```python
selected_df = df.select(df.old_col1, df.old_col2, new_col)
```
2. Alternatively, use the SQL string that follows `SELECT ... AS ...` protocal:
```python
selected_df = df.select('old_col1', 'old_col2', 'old_col1 * 2 - old_col2 AS new_col')
```

### Use `.groupBy()` method to create GroupedData and do aggregation

All of the common aggregation methods, like `.min()`, `.max()`, and `.count()` are `GroupedData` methods. These are created by calling the `.groupBy()` DataFrame method. For example, to find the minimum value of a column, col, in a DataFrame, df, you could do
```python
df.groupBy().min("col").show() # notice .groupBy() turns data into GroupedData object
```
This creates a GroupedData object (so you can use the `.min()` method), then finds the minimum value in col, and returns it as a DataFrame.

In [43]:
# examining the min timestamp
ratings.withColumn(
    'timestamp', ratings.timestamp.astype('int')).groupBy().min('timestamp').show()

+--------------+
|min(timestamp)|
+--------------+
|     828124615|
+--------------+



In [47]:
# examining the max rating
ratings.select(ratings.rating.astype('int')).alias(
    'rating').groupBy().max('rating').show()

+-----------+
|max(rating)|
+-----------+
|          5|
+-----------+



In [49]:
# examine the avg movie rating
ratings.withColumn('rating', ratings.rating.astype('int')).groupBy(
    'movieId').avg('rating').show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|    296|  4.09771986970684|
|   1090| 3.873015873015873|
| 115713| 3.642857142857143|
|   3210|3.3333333333333335|
|  88140|           3.34375|
|    829|2.6666666666666665|
|   2088| 2.388888888888889|
|   2294| 3.088888888888889|
|   4821|               2.8|
|  48738|               3.8|
|   3959|               3.5|
|  89864|3.4210526315789473|
|   2136|2.2857142857142856|
|    691|3.3333333333333335|
|   3606|               3.5|
| 121007|               4.0|
|   6731|              3.25|
|  27317|               3.5|
|  26082| 4.333333333333333|
| 100553|               4.0|
+-------+------------------+
only showing top 20 rows



In [61]:
# examine the count of ratings per user
ratings.groupBy('userId').count().show()

+------+-----+
|userId|count|
+------+-----+
|   296|   27|
|   467|   22|
|   125|  360|
|   451|   34|
|     7|  152|
|    51|  359|
|   124|   50|
|   447|   78|
|   591|   54|
|   307|  975|
|   475|  155|
|   574|   23|
|   169|  269|
|   205|   27|
|   334|  154|
|   544|   22|
|   577|  161|
|   581|   40|
|   272|   31|
|   442|   20|
+------+-----+
only showing top 20 rows



In addition to the GroupedData's built-in methods above, there is also the `.agg()` method. This method lets you pass an aggregate column expression that uses any of the aggregate functions from the `pyspark.sql.functions` submodule.

This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule **take the name of a column in a GroupedData table**.

In [62]:
# convention to import sql functions
import pyspark.sql.functions as F

In [80]:
# calculate the count of unique user, mean rating, rating standard deviation,
# of ratings by movieId
ratings.select('userId', 'movieId', ratings.rating.astype('float')).groupBy(
    'movieId').agg(F.avg('rating').alias('avg_rating'),
                   F.stddev('rating').alias('std_rating'),
                   F.countDistinct('userId').alias('user_count')).show()

+-------+------------------+-------------------+----------+
|movieId|        avg_rating|         std_rating|user_count|
+-------+------------------+-------------------+----------+
|   3210|3.4761904761904763| 0.8621610748180847|        42|
| 100553|               4.5|                0.0|         2|
|   3606|              3.75| 0.8660254037844386|         4|
|    296| 4.197068403908795| 0.9519971466349251|       307|
|   6731|             3.625|  1.026436275942851|         8|
|   2136|2.4642857142857144| 1.1513250846522238|        14|
|   1090| 3.984126984126984| 0.9374306596647389|        63|
|  48738|             3.975| 0.9101041004655513|        20|
|   2088|               2.5|  0.954863710632231|        18|
|  89864|3.6315789473684212| 0.5487892041970895|        19|
| 115713|3.9107142857142856|  1.178932123737814|        28|
|   2294|3.2444444444444445| 0.7659087162424525|        45|
|  88140|          3.546875| 0.6642892757944268|        32|
| 112911|               2.0| 1.471960144

### Use `spark_df.join()` to merge additional tables to a spark dataframe

The `.join()` method takes three arguments:

1. The first is the second spark dataFrame that you want to join with the first one (i.e. the "right" dataframe
2. The second argument, `on`, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table
3. The third argument, `how`, specifies the kind of join to perform

For instance:
```python
merge_df = left_df.join(right_df, 'foreign_key', 'leftouter')
```
Above code joines the right DataFrame to the left DataFrame by matching the `foreign_key` column values and will keep all records from the left DataFrame in the merged results

In [82]:
# merge the movies to the ratings dataframe
movies = my_spark.table('movies')
movie_rating = ratings.join(movies, 'movieId', 'leftouter')

In [84]:
movie_rating.show()

+-------+------+------+---------+----------+--------------------+--------------------+
|movieId|userId|rating|timestamp|rating_int|               title|              genres|
+-------+------+------+---------+----------+--------------------+--------------------+
|      1|     1|   4.0|964982703|         4|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|         4|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         4|         Heat (1995)|Action|Crime|Thri...|
|     47|     1|   5.0|964983815|         5|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   5.0|964982931|         5|Usual Suspects, T...|Crime|Mystery|Thr...|
|     70|     1|   3.0|964982400|         3|From Dusk Till Da...|Action|Comedy|Hor...|
|    101|     1|   5.0|964980868|         5|Bottle Rocket (1996)|Adventure|Comedy|...|
|    110|     1|   4.0|964982176|         4|   Braveheart (1995)|    Action|Drama|War|
|    151|     1|   5.0|964984041|         5